## Chapter 9. 파이썬스러운 객체

- 상속하지 않고도 덕 타이핑 매커니즘을 통해 사용자가 정의한 자료형도 자연스럽게 동작 가능

### 9.1 객체 표현 

- repr() : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환
- str() : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환
- repr()과 str() 메서드를 지원하려면 \_\_repr__()과 \_\_str__() 특별 메서드를 구현해야 한다.
- 객체를 표현하는 다른 방법을 지원하는 \_\_bytes__()와 \_\_format__() 두개의 특별 메서드도 존재

### 9.2 벡터 클래스의 부활 

In [6]:
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>

###### 예제 9-1 다양하게 표현되는 Vector2d 객체 

In [7]:
v1 = Vector2d(3,4)

In [8]:
v1.x, v1.y

(3.0, 4.0)

In [9]:
x, y= v1

In [10]:
x, y

(3.0, 4.0)

In [11]:
v1

Vector2d(3.0, 4.0)

In [12]:
v1_clone = eval(repr(v1))

In [13]:
v1 == v1_clone

True

In [14]:
print(v1)

(3.0, 4.0)


In [15]:
octets = bytes(v1)

In [16]:
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [17]:
abs(v1)

5.0

In [18]:
bool(v1), bool(Vector2d(0,0))

(True, False)

### 9.3 대안 생성자 

###### 예제 9-3

In [47]:
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # <4>
        return cls(*memv)
    
    def __format__(self, fmt_spec = ''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)

### 9.4 @classmethod와 @staticmethod 

- @classmethod : 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의
- @classmethod는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫 번쨰 인수로 받게 만든다.
  
- @staticmethod : 메서드가 특별한 첫 번째 인수를 받지 않도록 메서드를 변경. 본질적으로 정적 메서드는 모듈 대신 클래스 본체 안에 정의도니 평범한 함수일 뿐.

###### 예제 9-4 @classmethod와 @staticmethod의 동작 비교

In [20]:
class Demo():
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args
    

In [21]:
Demo.klassmeth()

(__main__.Demo,)

In [22]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [23]:
Demo.statmeth()

()

In [25]:
Demo.statmeth('spam')

('spam',)

### 9.5 포맷된 출력 

- format() 내장 함수와 str.format() 메서드는 실제 포맷 작업을 \_\_format__(format_spec)메서드에 위임한다.

In [26]:
brl = 1/2.43

In [27]:
brl

0.4115226337448559

In [28]:
format(brl, '0.4f')

'0.4115'

In [29]:
'1 BRL = {rate:0.2f} USD'.format(rate = brl)

'1 BRL = 0.41 USD'

In [30]:
format(42, 'b')

'101010'

In [31]:
format(2/3, '.1%')

'66.7%'

In [33]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'21:05:56'

In [34]:
"It's now {:%I:%M %p}".format(now)

"It's now 09:05 PM"

In [35]:
v1 = Vector2d(3,4)

In [36]:
format(v1)

'(3.0, 4.0)'

In [37]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [42]:
v1 = Vector2d(3,4)

In [43]:
format(v1)

'(3.0, 4.0)'

In [44]:
format(v1, '.2f')

'(3.00, 4.00)'

In [49]:
format(Vector2d(1,1), '3ep')

'(1.000000e+00, 1.000000e+00)'

### 9.6 해시 가능한 Vector2d

In [50]:
v1 = Vector2d(3,4)

In [51]:
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [56]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x) #언더바 2개를 붙이면 속성을 비공개로 바꾼다.
        self.__y = float(y)

    @property #프로퍼티의 게터(getter) 메서드를 나타냄
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [57]:
v1 = Vector2d(3,4)

In [58]:
v2 = Vector2d(3.1, 4.1)

In [59]:
hash(v1), hash(v2)

(7, 1031)

### 9.7 파이썬에서의 비공개 속성과 보호된 속성 

- 클래스의 속성명을 \_\_mood처럼 두개의 언더바로 시작하고 언더바 없이 또는 하나의 언더바로 끝나도록 정의하면, 파이썬은 언더바와 클래스병을 변수명 앞에 붙여 객체의 \_\_dict__에 저장한다

In [61]:
v1 = Vector2d(3,4)

In [62]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [64]:
v1._Vector2d__x

3.0

### 9.8 \_\_slots__클래스 속성으로 공간 절약하기 

- 기본적으로 파이썬은 객체 속성을 각 객체 안의 \_\_dict__라는 딕셔너리 속성에 저장한다.
- 딕셔너리는 빠른 접근 속도를 제공하기 위해 내부에 해시 테이블을 유지하므로, 메모리 사용량 부담이 상당히 크다.
- \_\_slots__ 클래스 속성을 이용해서 메모리 사용량을 엄청 줄일 수 있다. \_\_slots__ 속성을 파이썬 인터프리터가 객체 속성을 딕셔너리가 아닌 튜플에 저장하게 해준다

- 슈퍼클래스에서 상속받은 \_\_slots__ 속성은 서브클래스에 영향을 미치지 않는다. 각 클래스에서 개별적으로 정의된 \_\_slots__ 속성만 고려

- \_\_slots__를 정의하려면,클래스 속성을 생성하고 객체 속성 식별자들을 담은 문자열의 반복형을 할당한다.

###### 외부 예제

In [12]:
class SlotClass:
    __slots__ = ('name', ) #name이라는 객체 속성(?) 생성

class NoSlotClass:
    pass

slot = SlotClass()
no_slot = NoSlotClass()

no_slot.name = 'a'

In [15]:
slot.__dict__ # 에러 - slots은 dict을 가지고 있지 않기 때문에

AttributeError: 'SlotClass' object has no attribute '__dict__'

- slot 인스턴스는 slots을 사용하고있기 때문에 더 이상 네임 스페이스를 __dict__을 통해서 관리하지 않는다. 

In [7]:
no_slot.__dict__

{'name': 'a'}

In [17]:
import timeit

def check_time(obj):
    def inner():
        obj.name = 'hello'
        del obj.name
    return inner

print(min(timeit.repeat(check_time(slot), number=10000000))) # 1.6706519379999918
print(min(timeit.repeat(check_time(no_slot), number=10000000))) # 2.139917491999995

2.19532960000015
2.5732706999999664


-  인스턴스의 name 속성에 hello라는 문자열을 저장하는 작업을 천만번 수행하도록 했고 그 중에서 가장 짧은 시간이 출력되도록 했다.
- 결과는 큰 차이가 발생하진 않았지만 slots를 사용하지 않았을때 시간이 더 걸리는 것으로 보아 작업량이 더 많아 메모리를 더 점유한다고 생각할 수 있다.

##### 객체의 어트리뷰트를 출력하는 dir 함수를 통해 객체의 어트리뷰트들을 살펴보자

In [18]:
print(dir(slot))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'name']


In [19]:
print(dir(no_slot))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']


- slot 객체의 속성에는 dict와 weakref가 없다.

- 그러나 \_\_slots__ 리스트에 '\_\_dict__' 문자열을 추가하면 동적으로 속성을 생성할 수도 있게 해준다.
- 따라서  \_\_slots__을 사용하는 의미가 없어질 수 있다.

- 객체가 약한 참조를 지원하려면 \_\_weakref__ 속성이 필요하다.
- 이 속성은 기본적으로 존재하지만, 클래스가 \_\_slots__를 정의하고 이 클래스의 객체를 약한 참조의 대상이 되게 하려면 \_\_weakref__를 \_\_slots__리스트에 추가해야 한다.

### 9.8.1 \_\_slots__를 사용할 때 주의할 점

- 상속된 \_\_slots__속성을 무시하므로 각 클래스마다 \_\_slots__속성을 다시 정의해야한다.
- \_\_dict__를 \_\_slots__에 추가하지 않는 한 객체는 \_\_slots__에 나열된 속성만 가질 수 있다.
- \_\_weakref__를 \_\_slots__에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.

## 9.9 클래스 속성 오버라이드

- 메서드 오버라이딩(Method Overriding) : 부모 클래스로부터 상속받는 특정 메서드를 자식 클래스에서 재정의 하는 작업을 의미  
- 클래스 속성을 객체 속성의 기본값으로 사용하는 것이 파이썬의 특징.
- 다만, 존재하지 않는 객체 속성에 값을 저장하면, 새로운 객체속성을 생성하고 동일한 이름의 클래스 속성은 변경하지 않는다.
- 즉 각 객체가 서로 다른 속성을 갖도로 커스트마이징 할 수 있게 된다.

###### 외부 예제

In [24]:
class SportsTeam:
    def __init__ (self, team_name):
        self.team_name= team_name
        print(f"{self.team_name} 팀에 대한 정보입니다.")
        
    def won(self, won):
        print(f"{self.team_name} 팀은 {won}회 승리하였습니다.")
        
    def loss(self, loss):
        print(f"{self.team_name} 팀은 {loss}회 패배하였습니다.")
        
    def draw(self, draw):
        print(f"{self.team_name} 팀은 {draw}회 비겼습니다.")

In [25]:
class BasketballTeam(SportsTeam):
    def __init__ (self, team_name):
        self.team_name = team_name
    def draw(self, draw):
        print(f"{self.team_name} 팀은 {draw}회 연장전에 돌입했습니다.")

In [26]:
lal = SportsTeam('LA Lakers')
lal.draw(2)

LA Lakers 팀에 대한 정보입니다.
LA Lakers 팀은 2회 비겼습니다.


In [28]:
lal = BasketballTeam('LA Lakers')
lal.won(2)
lal.draw(2)

LA Lakers 팀은 2회 승리하였습니다.
LA Lakers 팀은 2회 연장전에 돌입했습니다.


- 만약 메서드 오버라이딩을 한 후, 부모 클래스의 메서드를 호출하고 싶다면 super()함수를 이용한다.